In [1]:
import pandas as pd
import geopandas as gpd
import re
from shapely.geometry import Polygon

In [2]:
df = pd.read_csv("taxable_values.csv",dtype=str)
df = df[0:37]
df['TAXING AUTHORITY'] = df['TAXING AUTHORITY'].str.replace('\d+','',regex=True)

In [3]:
gdf = gpd.read_file('Municipalitypoly_gdb_-3974701052066827405.geojson')

In [4]:
df['TAXING AUTHORITY'] = df['TAXING AUTHORITY'].str.strip()

In [5]:
gdf.columns

Index(['OBJECTID', 'MUNICUID', 'MUNICID', 'NAME', 'FIPSCODE', 'CREATEDBY',
       'CREATEDDATE', 'MODIFIEDBY', 'MODIFIEDDATE', 'GlobalID', 'geometry'],
      dtype='object')

In [6]:
gdf['NAME'] = gdf['NAME'].str.replace("UNINCORPORATED MIAMI-DADE","UNINCORPORATED")
gdf['NAME'] = gdf['NAME'].str.replace("INDIAN CREEK VILLAGE","INDIAN CREEK")

In [7]:
gdf = gdf.dissolve(by="NAME",as_index=False)

In [8]:
merged_gdf = pd.merge(left=df,left_on='TAXING AUTHORITY',right=gdf,right_on='NAME',how='left')

In [9]:
merged_gdf['TAXING AUTHORITY'] = merged_gdf['TAXING AUTHORITY'].str.title()
merged_gdf['TAXING AUTHORITY'] = merged_gdf['TAXING AUTHORITY'].str.replace('Mb','MB')
merged_gdf['TAXING AUTHORITY'] = merged_gdf['TAXING AUTHORITY'].str.replace('Dda','DDA')

In [10]:
empty_poly = Polygon()
merged_gdf["geometry"] = merged_gdf["geometry"].apply(
    lambda geom: geom if geom is not None else empty_poly
)

In [11]:
def compute_centroid(geom):
    if geom is None or geom.is_empty:
        return None
    return geom.centroid

merged_gdf["centroid"] = merged_gdf["geometry"].apply(compute_centroid)

# 3) (Optional) If you want separate latitude/longitude columns:
merged_gdf["centroid_x"] = merged_gdf["centroid"].apply(lambda pt: pt.x if pt is not None else None)
merged_gdf["centroid_y"] = merged_gdf["centroid"].apply(lambda pt: pt.y if pt is not None else None)

In [12]:
drop_columns = ['OBJECTID', 'MUNICUID',
       'MUNICID', 'NAME', 'FIPSCODE', 'CREATEDBY', 'CREATEDDATE', 'MODIFIEDBY',
       'MODIFIEDDATE', 'GlobalID']

In [13]:
merged_gdf = merged_gdf.drop(columns=drop_columns)

In [14]:
merged_gdf = gpd.GeoDataFrame(
    merged_gdf,
    geometry="geometry",
    crs="EPSG:4326"    
)



In [15]:
merged_gdf = merged_gdf.drop_duplicates(subset='TAXING AUTHORITY')

In [17]:
merged_gdf.to_file("merged_taxable_values.geojson", driver="GeoJSON")

In [18]:
import os

base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/miamidade_taxable_values
